#1: 기본 환경 설정 (Google Drive 연결 포함)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Aiffelthon

# Install MMCV and MMDetection using MIM.
!pip3 install -U openmim
!mim install mmcv-full==1.6.0
!mim install mmdet
# Install MMRotate from the source.
# !git clone https://github.com/open-mmlab/mmrotate.git
%cd /content/drive/MyDrive/Aiffelthon/mmrotate
!pip install -e .

import mmcv
from mmcv.runner import load_checkpoint
from mmcv import collect_env
collect_env()

# Check MMRotate installation
import mmrotate
from mmrotate.models import build_detector
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
from mmdet.apis import inference_detector, show_result_pyplot
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Give permission to scripts
!chmod 775 /content/drive/MyDrive/Aiffelthon/mmrotate/model.sh
!chmod 775 /content/drive/MyDrive/Aiffelthon/mmrotate/tools/train.py
!chmod 775 /content/drive/MyDrive/Aiffelthon/mmrotate/demo/image_demo.py

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 2.9 MB/s 
     |████████████████████████████████| 235 kB 10.5 MB/s 
     |████████████████████████████████| 51 kB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.1 MB 8.6 MB/s 
     |████████████████████████████████| 190 kB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Ai

#2: 폴더/파일명 설정 (모델별 수정필요)

In [20]:
# 1: 모델 CONFIG 파일 지정
# CFG_FILE = 'oriented_rcnn/oriented_rcnn_r50_fpn_1x_dota_le90_fair1m.py'
# CFG_FILE = 'rotated_faster_rcnn/rotated_faster_rcnn_r50_fpn_1x_dota_le90_fair1m.py'
# CFG_FILE = 'roi_trans/roi_trans_r50_fpn_1x_dota_le90_fair1m.py'
CFG_FILE = 'redet/redet_re50_refpn_1x_dota_le90_fair1m.py'

# 2: 모델 TRAIN 결과 폴더 지정 (mmrotate 폴더 아래)
# WORK_DIRS = 'work_dirs_fair1m_oriented-rcnn'
# WORK_DIRS = 'work_dirs_fair1m_rotated-faster-rcnn'
# WORK_DIRS = 'work_dirs_fair1m_roi-trans'
WORK_DIRS = 'work_dirs_fair1m_redet'

# 3: 모델 INFERENCE 결과 폴더 지정 (mmroate 폴더 아래)
# DEMO_DIRS = 'demo/fair1m_oriented-rcnn'
# DEMO_DIRS = 'demo/fair1m_rotated-faster-rcnn'
# DEMO_DIRS = 'demo/fair1m_roi-trans'
DEMO_DIRS = 'demo/fair1m_redet'

%cd /content/drive/MyDrive/Aiffelthon/mmrotate
!bash model.sh WORK_DIRS DEMO_DIRS CFG_FILE

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate


#3: TRAIN

##3-1: python script 파일 이용 (둘중 하나만 선택)

In [22]:
!./tools/train.py \
./configs/$CFG_FILE \
--work-dir=$WORK_DIRS

2022-08-27 01:38:50,363 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.12.1+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencod

##3-2: jupyter notebook 이용 (둘중 하나만 선택)

In [11]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
import torch.distributed as dist
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash
from mmdet import __version__
from mmdet.apis import init_random_seed, set_random_seed

from mmrotate.apis import train_detector
from mmrotate.datasets import build_dataset
from mmrotate.models import build_detector
from mmrotate.utils import collect_env, get_root_logger, setup_multi_processes

def parse_args():
    parser = argparse.ArgumentParser(description='Train a detector')
    parser.add_argument('config', help='train config file path')
    parser.add_argument('--work-dir', help='the dir to save logs and models')
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    parser.add_argument(
        '--auto-resume',
        action='store_true',
        help='resume from the latest checkpoint automatically')
    parser.add_argument(
        '--no-validate',
        action='store_true',
        help='whether not to evaluate the checkpoint during training')
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    parser.add_argument(
        '--diff-seed',
        action='store_true',
        help='Whether or not set different seeds for different ranks')
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file. If the value to '
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        'Note that the quotation marks are necessary and that no white space '
        'is allowed.')
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    parser.add_argument('--local_rank', type=int, default=0)
    args = parser.parse_args(['./configs/'+CFG_FILE, '--work-dir='+WORK_DIRS])
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    return args
    
%cd /content/drive/MyDrive/Aiffelthon/mmrotate

args = parse_args()

cfg = Config.fromfile(args.config)
if args.cfg_options is not None:
    cfg.merge_from_dict(args.cfg_options)

# set multi-process settings
setup_multi_processes(cfg)

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(args.config))[0])
if args.resume_from is not None:
    cfg.resume_from = args.resume_from
cfg.auto_resume = args.auto_resume
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
    if len(cfg.gpu_ids) > 1:
        warnings.warn(
            f'We treat {cfg.gpu_ids} as gpu-ids, and reset to '
            f'{cfg.gpu_ids[0:1]} as gpu-ids to avoid potential error in '
            'non-distribute training time.')
        cfg.gpu_ids = cfg.gpu_ids[0:1]
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
seed = init_random_seed(args.seed)
seed = seed + dist.get_rank() if args.diff_seed else seed
logger.info(f'Set random seed to {seed}, '
            f'deterministic: {args.deterministic}')
set_random_seed(seed, deterministic=args.deterministic)
cfg.seed = seed
meta['seed'] = seed
meta['exp_name'] = osp.basename(args.config)

model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=datasets[0].CLASSES)
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

/content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate


2022-08-27 00:17:38,985 - mmrotate - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla P100-PCIE-16GB
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: x86_64-linux-gnu-gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.12.1+cu113
PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencod

### **학습개시**

In [12]:
train_detector(
    model,
    datasets,
    cfg,
    distributed=distributed,
    validate=(not args.no_validate),
    timestamp=timestamp,
    meta=meta)

2022-08-27 00:17:46,667 - mmrotate - INFO - Start running, host: root@36e6391acf10, work_dir: /content/drive/.shortcut-targets-by-id/1YwVTiHBLdDN6lW5qFBt_FvhT8gncxmxj/Aiffelthon/mmrotate/work_dirs_fair1m_roi-trans
2022-08-27 00:17:46,669 - mmrotate - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 -------------------- 
after_train_it

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 7.0 task/s, elapsed: 1s, ETA:     0s

2022-08-27 00:18:22,217 - mmrotate - INFO - 
+-----------------+-----+------+--------+-------+
| class           | gts | dets | recall | ap    |
+-----------------+-----+------+--------+-------+
| PassengerShip   | 1   | 0    | 0.000  | 0.000 |
| Motorboat       | 2   | 0    | 0.000  | 0.000 |
| FishingBoat     | 0   | 0    | 0.000  | 0.000 |
| Tugboat         | 0   | 0    | 0.000  | 0.000 |
| other-ship      | 0   | 0    | 0.000  | 0.000 |
| EngineeringShip | 0   | 0    | 0.000  | 0.000 |
| LiquidCargoShip | 1   | 0    | 0.000  | 0.000 |
| DryCargoShip    | 5   | 0    | 0.000  | 0.000 |
| Warship         | 1   | 0    | 0.000  | 0.000 |
| SmallCar        | 23  | 100  | 0.000  | 0.000 |
| Bus             | 1   | 0    | 0.000  | 0.000 |
| CargoTruck      | 20  | 0    | 0.000  | 0.000 |
| DumpTruck       | 5   | 6    | 0.000  | 0.000 |
| other-vehicle   | 14  | 0    | 0.000  | 0.000 |
| Van             | 34  | 65   | 0.000  | 0.000 |
| Trailer         | 1   | 0    | 0.000  | 0.000 |
| Tra

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 7.7 task/s, elapsed: 1s, ETA:     0s

2022-08-27 00:18:56,723 - mmrotate - INFO - 
+-----------------+-----+------+--------+-------+
| class           | gts | dets | recall | ap    |
+-----------------+-----+------+--------+-------+
| PassengerShip   | 1   | 0    | 0.000  | 0.000 |
| Motorboat       | 2   | 0    | 0.000  | 0.000 |
| FishingBoat     | 0   | 0    | 0.000  | 0.000 |
| Tugboat         | 0   | 0    | 0.000  | 0.000 |
| other-ship      | 0   | 0    | 0.000  | 0.000 |
| EngineeringShip | 0   | 0    | 0.000  | 0.000 |
| LiquidCargoShip | 1   | 0    | 0.000  | 0.000 |
| DryCargoShip    | 5   | 0    | 0.000  | 0.000 |
| Warship         | 1   | 0    | 0.000  | 0.000 |
| SmallCar        | 23  | 124  | 0.043  | 0.002 |
| Bus             | 1   | 0    | 0.000  | 0.000 |
| CargoTruck      | 20  | 0    | 0.000  | 0.000 |
| DumpTruck       | 5   | 4    | 0.000  | 0.000 |
| other-vehicle   | 14  | 0    | 0.000  | 0.000 |
| Van             | 34  | 61   | 0.000  | 0.000 |
| Trailer         | 1   | 0    | 0.000  | 0.000 |
| Tra

#4: INTERFENCE

## 4-1: python script 파일 이용

In [8]:
!python demo/image_demo.py    \
./data/fair1m2.0/images_test/1213.tif     \
configs/$CFG_FILE    \
$WORK_DIRS/epoch_5.pth   \
--out-file=$DEMO_DIRS/fair1m_1213.jpg

/usr/local/lib/python3.7/dist-packages/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
load checkpoint from local path: work_dirs_fair1m_oriented-rcnn/epoch_5.pth
/usr/local/lib/python3.7/dist-packages/mmdet/models/dense_heads/anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '
